**Tugas 1**

In [1]:
# Contoh membuat DataFrame sederhana dan operasi dasar
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HandsOnPertemuan3').getOrCreate()

data = [('James', 'Sales', 3000),
        ('Michael', 'Sales', 4600),
        ('Robert', 'Sales', 4100),
        ('Maria', 'Finance', 3000)]
columns = ['EmployeeName', 'Department', 'Salary']

df = spark.createDataFrame(data, schema=columns)
df.show()

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|       James|     Sales|  3000|
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
|       Maria|   Finance|  3000|
+------------+----------+------+



**Tugas 2**



In [5]:
# Contoh operasi transformasi DataFrame
df.select('EmployeeName', 'Salary').show()
df.filter(df['Salary'] > 3000).show()
df.groupBy('Department').avg('Salary').show()

+------------+------+
|EmployeeName|Salary|
+------------+------+
|       James|  3000|
|     Michael|  4600|
|      Robert|  4100|
|       Maria|  3000|
+------------+------+

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
+------------+----------+------+

+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|     Sales|     3900.0|
|   Finance|     3000.0|
+----------+-----------+



**Tugas 3**

In [54]:
df.withColumn('SalaryBonus', df['Salary'] * 0.1).withColumn('TotalCompensation', df['Salary'] + F.col('SalaryBonus')).show()

+------------+----------+------+-----------+-----------------+
|EmployeeName|Department|Salary|SalaryBonus|TotalCompensation|
+------------+----------+------+-----------+-----------------+
|       James|     Sales|  3000|      300.0|           3300.0|
|     Michael|     Sales|  4600|      460.0|           5060.0|
|      Robert|     Sales|  4100|      410.0|           4510.0|
|       Maria|   Finance|  3000|      300.0|           3300.0|
+------------+----------+------+-----------+-----------------+



**Tugas 4**

In [55]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

windowSpec = Window.partitionBy('Department').orderBy('Salary')
df.withColumn('Rank', F.rank().over(windowSpec)).show()

+------------+----------+------+----+
|EmployeeName|Department|Salary|Rank|
+------------+----------+------+----+
|       Maria|   Finance|  3000|   1|
|       James|     Sales|  3000|   1|
|      Robert|     Sales|  4100|   2|
|     Michael|     Sales|  4600|   3|
+------------+----------+------+----+



**Tugas 5**

In [ ]:
from pyspark.sql import functions as sparkFuncs
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("tio").getOrCreate()

dataframe = spark.read.csv("Pendidikan.csv", header=True, inferSchema=True)
dataframe.show()

+--------------------+-------+-------+---------+-------------+----------------------------+-----------------------------+-----------------------+------------------------+-----------------+-----------------+-------------------------+-------------------------+------------------------+----+
|            Provinsi|Sekolah|  Siswa|Mengulang|Putus Sekolah|Kepala Sekolah dan Guru(<S1)|Kepala Sekolah dan Guru(≥ S1)|Tenaga Kependidikan(SM)|Tenaga Kependidikan(>SM)|Rombongan Belajar|Ruang kelas(baik)|Ruang kelas(rusak ringan)|Ruang kelas(rusak sedang)|Ruang kelas(rusak berat)|_c14|
+--------------------+-------+-------+---------+-------------+----------------------------+-----------------------------+-----------------------+------------------------+-----------------+-----------------+-------------------------+-------------------------+------------------------+----+
|Prov. D.K.I. Jakarta|   2224| 756455|     1202|          869|                        1089|                        38305|            

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

result = dataframe.select("Sekolah", "Siswa", "Putus Sekolah", "Mengulang") \
    .filter(F.col("Provinsi").isin("Prov. Jawa Tengah", "Prov. Jawa Barat", "Prov. Jawa Timur"))

result.show(truncate=False)

+-------+-------+-------------+---------+
|Sekolah|Siswa  |Putus Sekolah|Mengulang|
+-------+-------+-------------+---------+
|19553  |4487262|5675         |4778     |
|18618  |2595050|2399         |6909     |
|18980  |2530849|3327         |4694     |
+-------+-------+-------------+---------+



In [ ]:
result = dataframe.filter(F.col("Provinsi").isin(["Prov. Jawa Tengah", "Prov. Jawa Barat", "Prov. Jawa Timur"])) \
                 .groupBy("Provinsi") \
                 .agg(F.round(F.sum("siswa"), 2).alias("sum(siswa)"),
                      F.round(F.sum("Putus Sekolah"), 2).alias("sum(Putus Sekolah)"),
                      F.round(F.sum("Mengulang"), 2).alias("sum(Mengulang)"))

result.show(truncate=False)

+-----------------+----------+------------------+--------------+
|Provinsi         |sum(siswa)|sum(Putus Sekolah)|sum(Mengulang)|
+-----------------+----------+------------------+--------------+
|Prov. Jawa Timur |2530849   |3327              |4694          |
|Prov. Jawa Barat |4487262   |5675              |4778          |
|Prov. Jawa Tengah|2595050   |2399              |6909          |
+-----------------+----------+------------------+--------------+

